<a href="https://www.kaggle.com/code/shivrajpublic/google-simplilearn-project?scriptVersionId=112648248" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Steps to perform:

Load the data file using pandas.

In [ ]:
# import the library
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# import the data
data=pd.read_csv('/kaggle/input/googleplaystore1/googleplaystore.csv')

# Know the data

# First Code

In [ ]:
data.head()

In [ ]:
# print rows & col
data.shape

In [ ]:
# information of data
data.info()

# Check for null values in the data. Get the number of null values for each column.

# Drop records with nulls in any of the columns.

In [ ]:
data.isnull().sum()

In [ ]:
data.dropna(inplace=True)
data.isnull().sum()

Variables seem to have incorrect type and inconsistent formatting. You need to fix them: 

Size column has sizes in Kb as well as Mb. To analyze, you’ll need to convert these to numeric.

Extract the numeric value from the column

Multiply the value by 1,000, if size is mentioned in Mb

In [ ]:
data['Size']

In [ ]:
def change_size(size):
    if 'M' in size:
        x=size[:-1] # 19M
        x=float(x)*1000
        return x
    if 'k' in size: 
        x=size[:-1] #20k
        x=float(x)
        return x
    else:
        return None

In [ ]:
data['Size']=data['Size'].apply(change_size)

In [ ]:
data['Size']

In [ ]:
data.isnull().sum()

In [ ]:
data['Size'].fillna(method='ffill',inplace=True)

In [ ]:
data.isnull().sum()

# Reviews is a numeric field that is loaded as a string field. Convert it to numeric (int/float).

In [ ]:
data.head()

In [ ]:
data.dtypes

In [ ]:
data['Reviews'] = data['Reviews'].astype('int')

In [ ]:
data.dtypes

# Installs field is currently stored as string and has values like 1,000,000+. 

Treat 1,000,000+ as 1,000,000

remove ‘+’, ‘,’ from the field, convert it to integer

In [ ]:
data['Installs'] = data['Installs'].str.replace('+', '')
data['Installs'] = data['Installs'].str.replace(',', '')

In [ ]:
data['Installs'] = data['Installs'].astype('int')

In [ ]:
data.dtypes

In [ ]:
# Price field is a string and has $ symbol. Remove ‘$’ sign, and convert it to numeric.

In [ ]:
data['Price']=data['Price'].str.replace('$','')
data['Price']=data['Price'].astype('float')

In [ ]:
data.dtypes

Sanity checks:

Average rating should be between 1 and 5 as only these values are allowed on the play store. Drop the rows that have a value outside this range.

Reviews should not be more than installs as only those who installed can review the app. If there are any such records, drop them.

For free apps (type = “Free”), the price should not be >0. Drop any such rows.

In [ ]:
data[data['Rating']>5]

In [ ]:
data[data['Rating']<1]

In [ ]:
#Reviews should not be more than installs as only those who installed can review the app. 
#If there are any such records, drop them.

In [ ]:
len(data[data['Reviews'] > data['Installs']])

In [ ]:
data[data['Reviews'] > data['Installs']].index

In [ ]:
# drop these rows index
data.drop(data[data['Reviews'] > data['Installs']].index,inplace=True)

In [ ]:
len(data[data['Reviews'] > data['Installs']])

In [ ]:
data.shape

In [ ]:
# For free apps (type = “Free”), the price should not be >0. Drop any such rows
data[(data['Type'] == 'Free') & (data['Price'] > 0)]

Performing univariate analysis: 

Boxplot for Price

Are there any outliers? Think about the price of usual apps on Play Store.

Boxplot for Reviews

Are there any apps with very high number of reviews? Do the values seem right?

Histogram for Rating

How are the ratings distributed? Is it more toward higher ratings?

Histogram for Size

Note down your observations for the plots made above. Which of these seem to have outliers?

In [ ]:
sns.boxplot(x='Price',data=data)
plt.show()

In [ ]:
sns.boxplot(x='Reviews',data=data)
plt.show()

Histogram for Rating

How are the ratings distributed? Is it more toward higher ratings?

In [ ]:
sns.histplot(x='Rating',data=data)
plt.show()

Histogram for Size

Note down your observations for the plots made above. Which of these seem to have outliers?

In [ ]:
sns.histplot(x='Size',data=data)
plt.show()

 Outlier treatment: 

Price: From the box plot, it seems like there are some apps with very high price. A price of $200 for an application on the Play Store is very high and suspicious!

Check out the records with very high price

Is 200 indeed a high price?

Drop these as most seem to be junk apps

Reviews: Very few apps have very high number of reviews. These are all star apps that don’t help with the analysis and, in fact, will skew it. Drop records having more than 2 million reviews.

Installs:  There seems to be some outliers in this field too. Apps having very high number of installs should be dropped from the analysis.

Find out the different percentiles – 10, 25, 50, 70, 90, 95, 99

Decide a threshold as cutoff for outlier and drop records having values more than that

In [ ]:
data[data['Price']>200].size

In [ ]:
data.shape

In [ ]:
#Drop these as most seem to be junk apps
data.drop(data[data['Price']>200].index,inplace=True)

In [ ]:
data.shape

#Reviews: Very few apps have very high number of reviews. These are all star apps that don’t help with the analysis and, in fact, will skew it. Drop records having more than 2 million reviews.

In [ ]:
data[data['Reviews'] >= 2000000].shape

In [ ]:
data.drop(data[data['Reviews'] >= 2000000].index, inplace = True)

In [ ]:
data.shape

Installs: There seems to be some outliers in this field too. Apps having very high number of installs should be dropped from the analysis.

Find out the different percentiles – 10, 25, 50, 70, 90, 95, 99

Decide a threshold as cutoff for outlier and drop records having values more than that

In [ ]:
data['Installs'].quantile([0.1,.25,.5,.7,.9,.95,.99])

In [ ]:
data[data['Installs']>100000000.0].shape

In [ ]:
data.drop(data[data['Installs']>=100000000.0].index,inplace=True)

In [ ]:
data.shape

Bivariate analysis: Let’s look at how the available predictors relate to the variable of interest, i.e., our target variable rating. Make scatter plots (for numeric features) and box plots (for character features) to assess the relations between rating and the other features.

Make scatter plot/joinplot for Rating vs. Price

What pattern do you observe? Does rating increase with price?

Make scatter plot/joinplot for Rating vs. Size

Are heavier apps rated better?

Make scatter plot/joinplot for Rating vs. Reviews

Does more review mean a better rating always?

Make boxplot for Rating vs. Content Rating

Is there any difference in the ratings? Are some types liked better?

Make boxplot for Ratings vs. Category

Which genre has the best ratings?

For each of the plots above, note down your observation.

In [ ]:
sns.jointplot(x='Rating',y='Price',data=data)
plt.show()

In [ ]:
sns.jointplot(y='Rating',x='Size',data=data)
plt.show()

In [ ]:
# Make scatter plot/joinplot for Rating vs. Reviews

# Does more review mean a better rating always?

In [ ]:
sns.jointplot(x='Reviews',y='Rating',data=data)
plt.show()

Make boxplot for Rating vs. Content Rating

Is there any difference in the ratings? Are some types liked better?

In [ ]:
data.columns

In [ ]:
sns.boxplot(x='Rating',y='Content Rating',data=data)
plt.show()

Make boxplot for Ratings vs. Category

Which genre has the best ratings?

In [ ]:
plt.figure(figsize=(18,12))
sns.boxplot(x='Rating',y='Category',data=data)
plt.show()

Data preprocessing

For the steps below, create a copy of the dataframe to make all the edits. Name it inp1.

Reviews and Install have some values that are still relatively very high. Before building a linear regression model, you need to reduce the skew. Apply log transformation (np.log1p) to Reviews and Installs.

Drop columns App, Last Updated, Current Ver, and Android Ver. These variables are not useful for our task.

Get dummy columns for Category, Genres, and Content Rating. This needs to be done as the models do not understand categorical data, and all data should be numeric. Dummy encoding is one way to convert character fields to numeric. Name of dataframe should be inp2.

9. Train test split  and apply 70-30 split. Name the new dataframes df_train and df_test.

10. Separate the dataframes into X_train, y_train, X_test, and y_test.

11 . Model building

Use linear regression as the technique

Report the R2 on the train set

12. Make predictions on test set and report R2.

In [ ]:
data.head()

In [ ]:
# Data Preprocessing
#For the steps below, create a copy of the dataframe to make all the edits. Name it inp1.

In [ ]:
inp1=data.copy()

In [ ]:
inp1.columns

Reviews and Install have some values that are still relatively very high. Before building a linear regression model, you need to reduce the skew. Apply log transformation (np.log1p) to Reviews and Installs.

In [ ]:
inp1['Installs']=inp1['Installs'].apply(np.log1p)

In [ ]:
inp1['Reviews']=inp1['Reviews'].apply(np.log1p)

Drop columns App, Last Updated, Current Ver, and Android Ver. These variables are not useful for our task.

In [ ]:
inp1.drop(['App','Last Updated', 'Current Ver','Android Ver'],axis=1,inplace=True)

In [ ]:
inp1.shape

Get dummy columns for Category, Genres, and Content Rating. This needs to be done as the models do not understand categorical data, and all data should be numeric. Dummy encoding is one way to convert character fields to numeric. Name of dataframe should be inp2.

In [ ]:
inp1

In [ ]:
# convert categorical data into numeric 
# label encoding - blue black brown red yellow 0 1 2 3 4,   yes no 1 0
# dummy encoding- convert the data 0 and 1

# dummy variable/ One hot encoding
blue black brown red yellow
0     1     0     0    0
0     0     0     1    0
1     0     0     0    0

Get dummy columns for Category, Genres, and Content Rating. This needs to be done as the models do not understand categorical data, and all data should be numeric. Dummy encoding is one way to convert character fields to numeric. Name of dataframe should be inp2.

In [ ]:
inp2=pd.get_dummies(inp1)

In [ ]:
inp2.shape

In [ ]:
inp2

In [ ]:
# Extract features and target
y=inp2.pop('Rating')
X=inp2

In [ ]:
X

In [ ]:
y

In [ ]:
#Separate the dataframes into X_train, y_train, X_test, and y_test.
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=100)

In [ ]:
X_train.shape

In [ ]:
X_test.shape

In [ ]:
y_train.shape

In [ ]:
y_test.shape

In [ ]:
# Apply Linear Regression 
from sklearn.linear_model import LinearRegression # class
linear_reg=LinearRegression()

In [ ]:
linear_reg.fit(X_train,y_train) # object is learning LR on training data

In [ ]:
# pred 
y_pred=linear_reg.predict(X_test)

In [ ]:
y_test # actual outcome of x test

In [ ]:
y_pred # actual outcome of x test

In [ ]:
# print the error
from sklearn.metrics import mean_squared_error
print("MSE=",mean_squared_error(y_test,y_pred))

In [ ]:
# print the error
from sklearn.metrics import mean_squared_error
print("RMSE=",np.sqrt(mean_squared_error(y_test,y_pred)))

In [ ]:
# print r2 score
from sklearn.metrics import r2_score
print("R2 score=",r2_score(y_test,y_pred))

In [ ]:
# save the model
import joblib
joblib.dump(linear_reg,'Linear_Regression_model.sav')

In [ ]:
# load the model
model=joblib.load('Linear_Regression_model.sav')

In [ ]:
model.predict(X_test)